이것은 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 책의 노트북이다. 가독성을 위해 실행 가능한 코드 블록과 섹션 제목만 포함하고 책의 다른 모든 텍스트 단락, 그림 및 의사 코드는 생략한다.

**무슨 일이 일어나고 있는지 확인하고 싶다면 책과 함께 노트북을 사용하는 것을 권장한다.**

이 노트북은 TensorFlow 2.6 기반이다. (옮긴이: 버전 2.10.0, 애플 실리콘 M1 으로 테스트)

# 실제 세계에 대한 모범 사례

## 모델을 최대한 활용

### 하이퍼파라미터(Hyperparameter) 옵티마이저

#### KerasTuner 사용

**KerasTuner 모델-구축 함수**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**KerasTuner `HyperModel`**

In [3]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

/var/folders/nd/c88h9q6x4k1bgz9hsgds_2sr0000gn/T/ipykernel_70042/1594418805.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

Metal device set to: Apple M1 Pro


2022-11-17 16:10:46.917130: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-17 16:10:46.917397: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=10, # 100, 처리 속도가 약간 느리다
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 3 Complete [00h 01m 40s]
val_accuracy: 0.9743500351905823

Best val_accuracy So Far: 0.9743500351905823
Total elapsed time: 00h 05m 26s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
64                |64                |units
rmsprop           |rmsprop           |optimizer

Epoch 1/100


2022-11-17 16:16:23.458980: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 16:16:26.101055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 - 3s - loss: 0.4274 - accuracy: 0.8860 - val_loss: 0.2481 - val_accuracy: 0.9307 - 3s/epoch - 8ms/step
Epoch 2/100
391/391 - 3s - loss: 0.2256 - accuracy: 0.9355 - val_loss: 0.1863 - val_accuracy: 0.9501 - 3s/epoch - 7ms/step
Epoch 3/100
391/391 - 3s - loss: 0.1766 - accuracy: 0.9494 - val_loss: 0.1651 - val_accuracy: 0.9537 - 3s/epoch - 7ms/step
Epoch 4/100
391/391 - 3s - loss: 0.1452 - accuracy: 0.9582 - val_loss: 0.1468 - val_accuracy: 0.9578 - 3s/epoch - 7ms/step
Epoch 5/100
391/391 - 3s - loss: 0.1234 - accuracy: 0.9649 - val_loss: 0.1281 - val_accuracy: 0.9650 - 3s/epoch - 7ms/step
Epoch 6/100
391/391 - 3s - loss: 0.1067 - accuracy: 0.9696 - val_loss: 0.1181 - val_accuracy: 0.9676 - 3s/epoch - 7ms/step
Epoch 7/100
391/391 - 3s - loss: 0.0934 - accuracy: 0.9739 - val_loss: 0.1093 - val_accuracy: 0.9697 - 3s/epoch - 7ms/step
Epoch 8/100
391/391 - 3s - loss: 0.0827 - accuracy: 0.9767 - val_loss: 0.1046 - val_accuracy: 0.9706 - 3s/epoch - 7ms/step
Epoch 9/100
391/391 - 3s - l

2022-11-17 16:17:12.792186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 16:17:15.572242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 - 3s - loss: 0.4331 - accuracy: 0.8816 - val_loss: 0.2420 - val_accuracy: 0.9293 - 3s/epoch - 8ms/step
Epoch 2/100
391/391 - 3s - loss: 0.2202 - accuracy: 0.9374 - val_loss: 0.1775 - val_accuracy: 0.9500 - 3s/epoch - 8ms/step
Epoch 3/100
391/391 - 3s - loss: 0.1707 - accuracy: 0.9507 - val_loss: 0.1521 - val_accuracy: 0.9578 - 3s/epoch - 8ms/step
Epoch 4/100
391/391 - 3s - loss: 0.1409 - accuracy: 0.9589 - val_loss: 0.1387 - val_accuracy: 0.9591 - 3s/epoch - 7ms/step
Epoch 5/100
391/391 - 3s - loss: 0.1204 - accuracy: 0.9651 - val_loss: 0.1260 - val_accuracy: 0.9630 - 3s/epoch - 7ms/step
Epoch 6/100
391/391 - 3s - loss: 0.1046 - accuracy: 0.9705 - val_loss: 0.1143 - val_accuracy: 0.9661 - 3s/epoch - 7ms/step
Epoch 7/100
391/391 - 3s - loss: 0.0923 - accuracy: 0.9738 - val_loss: 0.1110 - val_accuracy: 0.9660 - 3s/epoch - 7ms/step
Epoch 8/100
391/391 - 3s - loss: 0.0828 - accuracy: 0.9762 - val_loss: 0.1065 - val_accuracy: 0.9693 - 3s/epoch - 7ms/step
Epoch 9/100
391/391 - 3s - l

KeyboardInterrupt: 

**최상의 하이퍼파라미터 구성 쿼리**

In [7]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [8]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [9]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
  7/391 [..............................] - ETA: 3s - loss: 1.9091 - accuracy: 0.4040  

2022-11-17 16:18:34.239384: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


389/391 [============================>.] - ETA: 0s - loss: 0.4165 - accuracy: 0.8885

2022-11-17 16:18:36.819561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 3s 7ms/step - loss: 0.4162 - accuracy: 0.8886 - val_loss: 0.2314 - val_accuracy: 0.9370
Epoch 2/100
391/391 [==============================] - 3s 7ms/step - loss: 0.2210 - accuracy: 0.9369 - val_loss: 0.1893 - val_accuracy: 0.9482
Epoch 3/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1745 - accuracy: 0.9493 - val_loss: 0.1640 - val_accuracy: 0.9536
Epoch 4/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1447 - accuracy: 0.9582 - val_loss: 0.1398 - val_accuracy: 0.9604
Epoch 5/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1231 - accuracy: 0.9644 - val_loss: 0.1275 - val_accuracy: 0.9624
Epoch 6/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1070 - accuracy: 0.9688 - val_loss: 0.1234 - val_accuracy: 0.9650
Epoch 7/100
391/391 [==============================] - 3s 7ms/step - loss: 0.0945 - accuracy: 0.9725 - val_loss: 0.1115 - val_accuracy: 0.9669
Epoch 8/100

NameError: name 'model' is not defined

In [0]:
best_models = tuner.get_best_models(top_n)

#### 올바른 검색 공간 구성의 예술

#### 파라미터 튜닝의 미래: 자동화된 기계 학습

### 모델 앙상블(ensembling)

## 수직 확장 모델 교육

### 혼합 정밀도로 GPU에서 훈련 속도 향상

#### 부동소수점 정밀도 (floating-point precision) 이해

In [12]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [13]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

#### 실제 혼합 정밀도 훈련

In [0]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### 다중-GPU 훈련

#### 2개 이상의 GPU을 건들기

#### 단일 호스트, 다중 장치를 동기적으로 훈련

### TPU 훈련

#### 구글 코랩(Google Colab)을 통해 TPU 사용

#### TPU 활용도 향상을 위한 단계 융합 활용

## 요약